In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
dataset = {
             "train_data" : "/content/drive/MyDrive/Colab Notebooks/archive/train",
             "valid_data" : "/content/drive/MyDrive/Colab Notebooks/archive/valid",
             "test_data" : "/content/drive/MyDrive/Colab Notebooks/archive/test"
          }
import os
import pandas as pd

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen=ImageDataGenerator(rescale=1.0/255,rotation_range=30,width_shift_range=10.0,height_shift_range=10.0,shear_range=10,zoom_range=0.1,vertical_flip=True)
valid_datagen=ImageDataGenerator(rescale=1.0/255)
test_datagen=ImageDataGenerator(rescale=1.0/255)

train_ds=train_datagen.flow_from_directory(dataset['train_data'],target_size=(224,224),batch_size=10)
valid_ds=valid_datagen.flow_from_directory(dataset['valid_data'],target_size=(224,224),batch_size=10)
test_ds=test_datagen.flow_from_directory(dataset['test_data'],target_size=(224,224),batch_size=10)

Found 489 images belonging to 3 classes.
Found 15 images belonging to 3 classes.
Found 15 images belonging to 3 classes.


In [ ]:
type(train_ds)

keras.preprocessing.image.DirectoryIterator

In [ ]:
import tensorflow as tf
model=tf.keras.models.Sequential(
    [
        tf.keras.layers.Conv2D(128,(3,3),activation='relu',input_shape=(224,224,3)),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(64,(3,3),activation='relu',input_shape=(224,224,3)),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(224,224,3)),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64,activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(48,activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(32,activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(3,activation='softmax')
    ]
)
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(train_ds,validation_data=valid_ds,epochs=100,batch_size=20)

Epoch 1/100
49/49 [==============================] - 12s 196ms/step - loss: 1.1092 - accuracy: 0.3456 - val_loss: 1.0995 - val_accuracy: 0.3333
Epoch 2/100
49/49 [==============================] - 8s 165ms/step - loss: 1.0880 - accuracy: 0.3804 - val_loss: 1.0990 - val_accuracy: 0.3333
Epoch 3/100
49/49 [==============================] - 8s 155ms/step - loss: 1.0927 - accuracy: 0.3845 - val_loss: 1.0797 - val_accuracy: 0.4000
Epoch 4/100
49/49 [==============================] - 9s 188ms/step - loss: 1.0836 - accuracy: 0.3845 - val_loss: 1.0525 - val_accuracy: 0.3333
Epoch 5/100
49/49 [==============================] - 8s 165ms/step - loss: 1.0153 - accuracy: 0.4519 - val_loss: 1.0207 - val_accuracy: 0.5333
Epoch 6/100
49/49 [==============================] - 8s 157ms/step - loss: 0.9776 - accuracy: 0.4683 - val_loss: 0.9703 - val_accuracy: 0.5333
Epoch 7/100
49/49 [==============================] - 9s 191ms/step - loss: 0.9734 - accuracy: 0.4888 - val_loss: 0.9643 - val_accuracy: 0.400

In [ ]:
# serialize model to JSON
#https://machinelearningmastery.com/save-load-keras-deep-learning-models/
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [ ]:
# load json and create model
from tensorflow.keras.models import model_from_json
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [ ]:
model_out = model.predict(test_ds)

2/2 [==============================] - 2s 1s/step


In [ ]:
model_out

array([[9.9731028e-01, 1.8752086e-06, 2.6878628e-03],
       [9.8966366e-01, 1.8702433e-07, 1.0336068e-02],
       [5.7094824e-02, 3.3267531e-01, 6.1022985e-01],
       [9.9902487e-01, 1.9736161e-08, 9.7518793e-04],
       [5.1246719e-03, 2.5585300e-04, 9.9461949e-01],
       [9.9998486e-01, 1.3902085e-13, 1.5086112e-05],
       [7.5728334e-02, 6.1874115e-01, 3.0553055e-01],
       [6.1106075e-02, 2.9893516e-02, 9.0900034e-01],
       [7.5890100e-01, 2.1289112e-03, 2.3897007e-01],
       [1.4004753e-03, 8.3928531e-01, 1.5931420e-01],
       [3.7388831e-02, 4.1165534e-02, 9.2144567e-01],
       [4.7390438e-03, 9.0964341e-01, 8.5617557e-02],
       [8.0218382e-02, 4.8816991e-01, 4.3161175e-01],
       [8.2784193e-03, 3.9773440e-01, 5.9398723e-01],
       [3.6584216e-03, 2.3638494e-03, 9.9397767e-01]], dtype=float32)

In [ ]:
y_pred_labels = np.argmax(model_out, axis=1)

In [ ]:
accuracy = accuracy_score(y_test_labels, y_pred_labels)
confusion_mat = confusion_matrix(y_test_labels, y_pred_labels)

print("Accuracy:", accuracy)
print("Confusion Matrix:")
print(confusion_mat.numpy())

plt.plot(history.history['loss'])
plt.title('Model Convergence')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

In [ ]:
# dv in deep learning
# https://www.pluralsight.com/guides/data-visualization-deep-learning-model-using-matplotlib

In [ ]:
loaded_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
score = loaded_model.evaluate(test_ds, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

accuracy: 73.33%


In [ ]:
import matplotlib.pyplot as plt


In [ ]:
#hyper-parameter tunning
def customizable_model(train_data,valid_data,test_data,act='relu',drop_rate=0.3,epochs=20,l_rate=0.001):
  import tensorflow as tf
  model=tf.keras.models.Sequential(
      [
          tf.keras.layers.Conv2D(128,(3,3),activation='relu',input_shape=(224,224,3)),
          tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
          tf.keras.layers.Conv2D(64,(3,3),activation=act),
          tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
          tf.keras.layers.Dropout(drop_rate),
          tf.keras.layers.Conv2D(32,(3,3),activation=act),
          tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
          tf.keras.layers.Flatten(),
          tf.keras.layers.Dense(64,activation=act),
          tf.keras.layers.Dropout(drop_rate),
          tf.keras.layers.Dense(48,activation=act),
          tf.keras.layers.Dropout(drop_rate),
          tf.keras.layers.Dense(32,activation=act),
          tf.keras.layers.Dropout(drop_rate),
          tf.keras.layers.Dense(3,activation='softmax')
      ]
  )
  model.compile(optimizer=tf.keras.optimizers.Adam(
    learning_rate=l_rate),loss='categorical_crossentropy',metrics=['accuracy'])
  history=model.fit(train_data,validation_data=valid_data,epochs=epochs,batch_size=20)

  model_json = model.to_json()
  with open("model.json", "w") as json_file:
      json_file.write(model_json)
  # serialize weights to HDF5
  model.save_weights("model.h5")
  print("Saved model to disk")

  return history

In [ ]:
# https://www.gradio.app/docs/interface